In [1]:
import numpy as np
import os, sys
import matplotlib.pyplot as plt

# basedir = os.path.dirname(os.getcwd())
basedir = os.path.abspath(os.path.join(os.getcwd() ,"../"))
_py = os.path.join(basedir, 'py')
_data = os.path.join(basedir, 'data')

sys.path.insert(1, _py)
import loads
import lia
import ray as rayt
import lad
import figures

import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

%matplotlib qt


# Data structure

```
root
│   requirements.yml
│   readme.md  
│
└───data
│   └───test
│       │   s0100000.numpy
│       │   s0200000.numpy
│       │   ...
│       │   mesh.ply
│       │   scanner_pos.txt
│   
└───py
    │   loads.py
    │   lia.py
    │   lad.py
    │   ray.py
    │   figures.py

```

## Blensor output transformation

Most of the functions used in this chapter need:

``` Python
import loads
```

In order to get the `LIA` and hence the `LAD`, we need to segmentated the trees and the leaves.

First, we define the name of the directory where the Blensor output data is, in this particular case we will look for directory `test`. Pipeline will look for this directory inside the `data` directory.

In [2]:
mockname = 'test'

Next, we convert Blensor output `txt` files that have fake `numpy` extension to real `npy`. This is done trhough function `loads.numpy2npy()` as shown below,

In [3]:
loads.numpy2npy(mockname)

s0500000.numpy done --> Number of beams: 22500
s0200000.numpy done --> Number of beams: 22500
s0700000.numpy done --> Number of beams: 22500
s0400000.numpy done --> Number of beams: 22500
s0100000.numpy done --> Number of beams: 22500
s0600000.numpy done --> Number of beams: 22500
s0300000.numpy done --> Number of beams: 22500


```{note}
Transforming to `npy` reduce the size of files, besides is much faster to load than the Blensor `txt` output files.
```

The structure looks like,

```
root
|
└───data
    └───test
        │   s0100000.numpy
        │   s0200000.numpy
        │   ...
        │   mesh.ply
        │   scanner_pos.txt
        │   s0100000.npy
        │   s0200000.npy
        │   ...
```

## Tree and leaves segmentation

Now we create the module to segmentate trees. This will be tuned acordingly for each data set, so below module only works for this particular data set.

In [3]:
def segtree(df, leaves, show=False):

    trees = {}

    if show:
        plt.figure(figsize=(14, 8))

    # centres
    x, y = [0], [0]
    num = 0
    dx, dy = 5, 5

    for i in x:
        for j in y:
            
            keep = np.ones(len(df['x']), dtype=bool)
            keep &= (df['x'] < i+dx) & (df['x'] > i-dx)
            keep &= (df['y'] < j+dy) & (df['y'] > j-dy)

            trees['tree_%s' %(str(num))] = keep
            
            if show:
                plt.scatter(df['x'][leaves & keep], df['y'][leaves & keep], s=0.5, label=num)
                        
            num += 1

    if show:
        plt.legend()
    
    return trees


We segmentate the trees below,

In [4]:
# load data into a pandas data frame
df = loads.npy2pandas(mockname)
# extract leaves. Boolean array output
leaves = loads.extract_leaves(df, show=False)
# extract trees. Dictionary with boolean arrays output
trees = segtree(df, leaves)

That's it! So what we just did? First, with function `loads.npy2pandas()` we load all the `npy` files into a pandas DataFrame (`DF`) and we add three more columns with the $x$, $y$, and $z$ positions of the sensors that are stored in file `scanner_pos.txt`. Then, since this is a mockup dataset, we can easily separate the leaves from everythin else in the point cloud (`PC`). We do this with function `loads.extract_leaves()` that requires the pandas `DF` as input. Finally, we invoke the above module to segmentate trees that requires the pandas `DF` as well.

outputs from this are:

- `df`: Pandas DF with the entire PC
- `leaves`: numpy boolean array of PC dimensions with True for Points concerning leaves only
- `trees`: python dictionary where each entry contains one tree in the form of boolena array with PC dimensions

Below piece of code shows an example of how to visualize the leaves points from one tree only, this will be know as the `Leaves Point Cloud` (LPC) and is shown in Fig. {numref}`lpc`.

Below piece of code shows an example of how to visualize the leaves points from one tree only.

In [6]:
# show the point cloud from leaves of firs tree only
keep = (trees['tree_0']) & (leaves)
loads.showPCfromDF(df[keep])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In the subsequent chapters we will be comparing our estimations with the *True* values using the `mesh.ply` file located in the root directory `test`. The following piece of code shows how we can visualize this mesh that requires importing the library `lad`. Fig. {numref}`mesh` show this mesh.

In [7]:
import lad
meshfile = lad.get_meshfile(mockname)
lad.see_mesh(meshfile)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


# `Leaf Inclination Angle` (LIA) estimation

## Intro

Most functions used in this chapter are in library:

``` Python
import lia
```

Outputs will be placed inside directory `lia`,

```
root
└───data
    └───test
        │   s0100000.numpy
        │   s0200000.numpy
        │   ...
        │   mesh.ply
        │   scanner_pos.txt
        │   s0100000.npy
        │   s0200000.npy
        │   ...
        └───lia
            │   angles_<treename>.npy
            │   weights_<treename>.npy
            │   leaf_angle_dist_<treename>.png
            │   leaf_angle_dist_height_<treename>.png
            │   bestfits_pars_treename>.png

```

## The method (`lia.leaf_angle()`)

The main function that computes the LIA is `lia.leaf_angle()` which uses a KDtree approximation. The steps are as follow: 

1. `Compute normals`: This method fits a plane based on the nearesth neighbors for each point and gets the normal of this plane.
    
2. `Compute zenith angles`: Then, using the dot product with get the angle with respect to the zenith (i.e. agains vector (0, 0, 1)) 
    
3. `Range correction`: The results angles run from $0 < \theta < 180$, however we require these to be in the range $0 < \theta < 90$ therefore we transfom those angles $> 90$ with relation:

```{math}
:label: angcorr
\theta_{L} = 180 - \theta
```

4. `Weights correction`: The resulting LIA is biased to PC density and completeness. In order to reduce this biases, we compute weights via voxelization,

```{math}
:label:
\eta_{i} = n_{i}/L^{3} \\
  \bar{\eta} = \frac{1}{N}\sum_{i=0}^{N} \eta{i}
```

where $n_{i}$ is the number of points within voxel $i$, $L$ is the voxel size, and $N$ is the total number of voxels, then $\eta_{i}$ is the volume density of voxel $i$, and $\bar{\eta}$ is the mean volume density.

The function `lia.leaf_angle()` has to be ran per tree and requieres 6 input parameters:

- `points`: $x$, $y$ and $z$ coordinates of the leaf point cloud (LPC).
- `mockname`: name of directory where the data is.
- `treename`: name/index of tree.
- `voxel_size_w`: voxel size for `weights correction` i.e. $L$.
- `kd3_sr`: KDtree searching radius for the nearest neighboors serch.
- `max_nn`: Maximum number of nearest neightbors to be considered.

This function returns a set of files inside directory `lia`:

- `angles_<treename>.npy`: LIA for the LPC. One file per tree.
- `weights_<treename>.npy`: LIA weights for the LPC. One file per tree.
- `leaf_angle_dist_<treename>.png`: Figure of LIA ($\theta_{L}$) distribution with `weights correction`. If `Truth` LIA available, this will be shown alongside. One figure per tree.
- `leaf_angle_dist_height_<treename>.png`: Top - Figure of LPC distribution accross different heights in terms of voxels $k$. Bottom - If `Truth` LIA available, $\theta_{L}^{truth} - \theta_{L}$. The different curves show this for different heights ($k$). One figure per tree.

## Look for best-fit `voxel_size_w`, `kd3_sr` and `max_nn` with `lia.bestfit_pars_la()`

If truth LIA available i.e. there's a mesh file `mesh.ply` in the `test` directory, then we will be able to run function `lia.bestfit_pars_la()` which essentialy runs `lia.leaf_angle()` for a range of values in `voxel_size_w`, `kd3_sr` and `max_nn` and find the best-fit for these three based on the minimal $\chi^{2}$ between the estimated LIA and the truth LIA.

`lia.bestfit_pars_la()` is as well ran per tree and requires only `points`, `mockname` and `treename`. It returns `bestfit_<treename>.npy` file that contains the `voxel_size_w`, `kd3_sr` and `max_nn` best-fit values per tree. it also returns a dictionary with the $\chi^{2}$ for each of these runs.

Using output dictionary from `bestfit_pars_la` we can run `bestfit_pars_la` to create figure `bestfits_pars_treename>.png` that shows the $\chi^{2}$ for all the ranges used in `voxel_size_w`, `kd3_sr` and `max_nn`.

```{admonition} To-Do
:class: important
Current LIA implementation works without `Truth` LIA, however, we need it to estimate the best-fits `voxel_size_w`, `kd3_sr` and `max_nn` parameters. We need to find the relation between these three and LPC that could rely on the LPC density, leaf size, leaf area, etc.
```

The piece of code bellow runs `lia.bestfit_pars_la()` and `lia.best_fit_pars_plot()` for each tree.


In [9]:
for key, val in trees.items():

    keep = (val) & (leaves) # take the LPC per tree
    df_ = df[['x', 'y', 'z']][keep]
    points = loads.DF2array(df_)
    res = lia.bestfit_pars_la(points, mockname, treename=key)
    lia.best_fit_pars_plot(res, key, mockname)


voxel_size_w 0.0001 DONE...
voxel_size_w 0.001 DONE...
voxel_size_w 0.01 DONE...
voxel_size_w 0.1 DONE...
voxel_size_w 1 DONE...
voxel_size_w BESTFIT:	 0.01
kd3_sr 0.001 DONE...
kd3_sr 0.01 DONE...
kd3_sr 0.1 DONE...
kd3_sr 1.0 DONE...
kd3_sr BESTFIT:	 1.0
max_nn 3 DONE...
max_nn 5 DONE...
max_nn 10 DONE...
max_nn 20 DONE...
max_nn 50 DONE...
max_nn 100 DONE...
max_nn BESTFIT:	 5


once we find the best-fit parameters we get figure `bestfits_pars_treename>.png` that is shown in Fig. {numref}`bestfits_pars`. we use these best fits to run `lia.leaf_angle()` and get the LIA and corresponding weigths per tree. The code that does that is shown below and in Fig. {numref}`lia_dist` we show `leaf_angle_dist_<treename>.png` and in Fig. {numref}`lia_dist_h` `leaf_angle_dist_height_<treename>.png`.

In [10]:
# load bestfit results
for key, val in trees.items():

    keep = (val) & (leaves)
    df_ = df[['x', 'y', 'z']][keep]
    points = loads.DF2array(df_)

    bestfit_file = os.path.join(_data, mockname, 'lia', 'bestfit_%s.npy' %(key))
    res = np.load(bestfit_file, allow_pickle=True)
    res = res.tolist()

    text = 'leaf area=%.2f \n %s=%.4f \n %s=%.4f \n %s=%.4f ' %(res['leafsize'], 'voxel_size_w', res['voxel_size_w_bestfit'],'kd3_sr', res['kd3_sr_bestfit'],'max_nn', res['max_nn_bestfit'])
    print(text)

    chi2 = lia.leaf_angle(points, mockname, key, res['voxel_size_w_bestfit'], 
                            res['kd3_sr_bestfit'], res['max_nn_bestfit'], save=True,
                                savefig=True, text=text)
                                

leaf area=0.04 
 voxel_size_w=0.0100 
 kd3_sr=1.0000 
 max_nn=5.0000 


# `Leaf Area Density` (LAD) estimation

## Intro

Most functions used in this chapter are in library:

``` Python
import lad
```

The LAD method implemented here uses the `Voxel 3D contact-bases frecuency` method first introduced by `HOSOI AND OMASA: VOXEL-BASED 3-D MODELING OF INDIVIDUAL TREES FOR ESTIMATING LAD`.

The model looks like:

```{math}
:label:
LAD(h, \Delta H) = \frac{1}{\Delta H} \sum_{k=m_{h}}^{m_{h}+\Delta H} l(k),
```

where,

```{math}
:label:
l(k) = \alpha(\theta)N(k) \\
    = \alpha(\theta) \cdot \frac{n_{I}(k)}{n_{I}(k) + n_{P}(k)}.
```

$l(k)$ is the `Leaf Area Index` (LAI) of the kth horizontal layer of the voxel array within a plant region, $\Delta H$ is the horizontal layer thickness, and $m_{h}$ and $m_{h}+\Delta H$ are the voxel coordinates on the vertical axis equivalent to height $h$ and $h+\Delta H$ in orthogonal coordinates ($h = \Delta k \times m_{h}$). The LAI of the kth horizontal layer $l(k)$ is the product of the contact frequency $N(k)$ of laser beams in the kth layer and the coefficient $\alpha(\theta)$, which corrects for leaf inclination at laser incident zenith angle $\theta$.

$n_{I}(k)$ is the number of voxels where the laser beams is intercepted by the kth layer, $n_{P}(k)$ is the number of voxels where the laser beams passed through the kth layer, and $n_{I}(k) + n_{P}(k)$ is the total number of voxels where the incident laser beams reach the kth layer.

Despite the complexity of this method, it requieres only one parameter, the `voxel_size`. We will introduce a second parameter, the `downsample` whose importance will be explained later. The main steps towards LAD estimation are:

1. Computing $n_{P}(k)$
2. Computing $n_{I}(k)$
3. Computing $\alpha(\theta)$
4. Estimate LAD

For this example, we will usea a `voxel_size` = 0.2 and `downsample` = 0.05 which means that we downsample our whole data to only $5\%$. A reminder that as well as in for the LIA, the following process is per tree.

In [181]:
downsample = 0.1
voxel_size = 0.1
# to check everything looks fine
show = False
sample = None

## Computing $n_{P}(k)$

Seeing where the beam pass through in the voxelize `Plant Region` (PR) is a tipycal ray tracing problem and it's reduced to see whether the ray hit or not an axis align bounding box (AABB).

The below module grabs the requiered downsample percentge of the data with a random subsample and if it's the first time we ran this, it will create the directory `lad_<downsmple>`. All the subsequent results will be stored inside this directory. The first time a particular downsample is ran, it will store the file `inds.npy` containing a boolean array with size of the pandas DF where True being the selected random subsample requested. If we change the `voxel_size` but not the `downsample`, then the below module will look first for the `inds.npy` instead of searching for another random subsample, this to maintain uniformity between different voxels sizes approaches.

The function `main` does the magic here, it has to be ran per tree and requires 6 input parameters:

- `points`: $x$, $y$ and $z$ coordinates from the downsample data in the form of numpy array.
- `sensors`: $x$, $y$ and $z$ coordinates of sensor responsible from each point in `points` parameter above.
- `pointsPR`: `points` above filtered to the LPC.
- `voxel_size`: Voxel Size.
- `resdir`: Name of output directory for the specific `downsample`.
- `treename`: Name/index of tree.

```{note}
`pointsPR` is require to get the same voxelization dimensions as in $n_{I}$.
```
This function returns two files:

- `m3s_<treename>_<voxel_size>.npy`: numpy boolean 3D-array with number of voxels dimensions. True if a beam hit the voxel.
- `m3count_<treename>_<voxel_size>.npy`: numpy 3D-array with number of voxels dimensions. Each entry contains the number of beams that passed trhough that voxel.

```{admonition} To-Do
:class: important
Note that this is the slowest module of the entire pipeline, taking up to 5 minutes for a sample of 10,000 beams. This can be improved easlily if binding with a `C++` ray AABB module instead.
```

Below we show the piece of code that computes this,

In [5]:

POINTS = loads.DF2array(df[['x', 'y', 'z']])
SENSORS = loads.DF2array(df[['sx', 'sy', 'sz']])

def get_rays(downsample, voxel_size, sample=None, show=False):

    if downsample is not None:

        resdir = os.path.join(_data, mockname, 'lad_%s' %(str(downsample)))
        if not os.path.exists(resdir):
            os.makedirs(resdir)

        outdir = os.path.join(resdir, 'inds.npy')
        if os.path.exists(outdir):
            print('inds file already exists for donwnsample of %.3f at %s' %(downsample, outdir))

            inds = np.load(outdir)

            points = POINTS[inds]
            sensors = SENSORS[inds]

        else:

            print('inds not been created yet for donwnsample of %.3f' %(downsample))
            idx = np.random.randint(0, len(df), int(len(df) * downsample))
            inds = np.zeros(len(df), dtype=bool)
            inds[idx] = True

            points = POINTS[inds]
            sensors = SENSORS[inds]

            np.save(outdir, inds)

    else:

        resdir = os.path.join(_data, mockname, 'lad')
        if not os.path.exists(resdir):
            os.makedirs(resdir)

    if sample is not None:

        idx = np.random.randint(0, len(df), int(sample))
        points = POINTS[idx]
        sensors = SENSORS[idx]

    for key, val in trees.items():

        inPR = (val) & (leaves) & (inds)
        pointsPR = POINTS[inPR]
        m3s = rayt.main(points, sensors, pointsPR, voxel_size, resdir, key, show=show)
    

In [535]:
downsample = 0.05
voxel_size = 0.2
sample = None
show = False

get_rays(downsample, voxel_size, sample, show)

inds file already exists for donwnsample of 0.050 at /Users/omar/projects/planttech/data/test/lad_0.05/inds.npy
max --> [45, 46, 29]
min --> [0, 0, 0]


7672it [01:55, 66.20it/s] 

tot vox: 	 64860
voxels hitted: 	 48100
Percentage of voxels hitted by beam: 0.74
voxels hitted (OLD): 	 55731
Percentage of voxels hitted by beam (OLD): 0.86


In [352]:
downsample = 0.2
voxel_size = 0.13

get_rays(downsample, voxel_size)

inds file already exists for donwnsample of 0.200 at /Users/omar/projects/planttech/data/test/lad_0.2/inds.npy
max --> [72, 70, 45]
min --> [0, 0, 0]


28463it [09:54, 47.87it/s] 

tot vox: 	 238418
voxels hitted: 	 214734
Percentage of voxels hitted by beam: 0.90


In [353]:
downsample = 0.3
voxel_size = 0.11

get_rays(downsample, voxel_size)

inds file already exists for donwnsample of 0.300 at /Users/omar/projects/planttech/data/test/lad_0.3/inds.npy
max --> [85, 83, 54]
min --> [0, 0, 0]


40779it [17:18, 39.27it/s] 

tot vox: 	 397320
voxels hitted: 	 359508
Percentage of voxels hitted by beam: 0.90


In [540]:
for DS in [0.01, 0.05, 0.1, 0.15, 0.2, 0.3]:
    for VS in [0.1, 0.15, 0.2, 0.3, 0.4]:

        get_rays(DS, VS)

inds file already exists for donwnsample of 0.010 at /Users/omar/projects/planttech/data/test/lad_0.01/inds.npy
max --> [89, 80, 57]
min --> [0, 0, 0]


1564it [00:38, 41.05it/s]


tot vox: 	 422820
voxels hitted: 	 64972
Percentage of voxels hitted by beam: 0.15
voxels hitted (OLD): 	 66795
Percentage of voxels hitted by beam (OLD): 0.16
inds file already exists for donwnsample of 0.010 at /Users/omar/projects/planttech/data/test/lad_0.01/inds.npy
max --> [59, 53, 38]
min --> [0, 0, 0]


1564it [00:21, 71.46it/s]


tot vox: 	 126360
voxels hitted: 	 37307
Percentage of voxels hitted by beam: 0.30
voxels hitted (OLD): 	 39705
Percentage of voxels hitted by beam (OLD): 0.31
inds file already exists for donwnsample of 0.010 at /Users/omar/projects/planttech/data/test/lad_0.01/inds.npy
max --> [45, 40, 29]
min --> [0, 0, 0]


...

tot vox: 	 69936
voxels hitted: 	 55046
Percentage of voxels hitted by beam: 0.79
voxels hitted (OLD): 	 66740
Percentage of voxels hitted by beam (OLD): 0.95
inds file already exists for donwnsample of 0.300 at /Users/omar/projects/planttech/data/test/lad_0.3/inds.npy
max --> [31, 31, 20]
min --> [0, 0, 0]


40779it [05:47, 117.24it/s]


tot vox: 	 21504
voxels hitted: 	 15736
Percentage of voxels hitted by beam: 0.73
voxels hitted (OLD): 	 20844
Percentage of voxels hitted by beam (OLD): 0.97
inds file already exists for donwnsample of 0.300 at /Users/omar/projects/planttech/data/test/lad_0.3/inds.npy
max --> [23, 23, 15]
min --> [0, 0, 0]


40779it [05:19, 127.75it/s]

tot vox: 	 9216
voxels hitted: 	 6333
Percentage of voxels hitted by beam: 0.69
voxels hitted (OLD): 	 9049
Percentage of voxels hitted by beam (OLD): 0.98


## Computing $n_{I}(k)$

The $n_{I}$ per voxel is computed in function `lad.compute_attributes()`. It essentialy voxelize the LPC to get the PR dimensions (which have to be the same as in $n_{P}$). Then, for a numpy boolean 3D-array with voxelize PR dimensions, we fill it with True if there's a point in the voxel.

This function looks for previous `m3s_<treename>_<voxel_size>.npy` result and get the attributes in the form of the same size numpy 3D-array (`m3att`). The attributes are:

- 1 if any LPC in that voxel
- 2 if any beam pass trhough that  voxel
- 3 if none of previous

It requires 4 input parameters `pointsPR`, `resdir`, `voxel_size`, `treename` which were defined in section {ref}`sec:np`. It returns the attributes numpy 3D-array.

## Computing $\alpha(\theta)$

$\alpha(\theta)$ is expressed in terms of $G(\theta)$, 

```{math}
:label:
\alpha(\theta) = \frac{\cos(\theta)}{G(\theta)},
```

where $G(\theta)$ is the mean projection of a unit leaf area on a plane perpendicular to the direction of the laser beam. This quantity is determined with the assumption that leaves are positioned symmetrically with respect to the azimuth anc can be represented as:

```{math}
:label:
G(\theta) =  \sum_{q=1}^{T_{q}} g(q) S(\theta, \theta_{L}(q))
```

where $S(\theta, \theta_{L}(q))$ is expresed in terms of the leaf inclination angle (LIA) $\theta_{L}$ (the zenith angle of the normal to the leaf surface), and $\theta$ is the laser-beam incident zenith angle:

```{math}
:label:
S(\theta, \theta_{L}) = \cos\theta \cos \theta_{L}, \hspace{.5cm} \textrm{for } \theta \leq \pi/2 - \theta_{L}
```

```{math}
:label:
S(\theta, \theta_{L}) = \cos\theta \cos \theta_{L} \left[ 1 + \frac{2}{\pi}(\tan x - x) \right], \hspace{.5cm} \textrm{for } \theta \gt \pi/2 - \theta_{L}
```

```{math}
:label:
x = \cos^{-1}\left( \cot \theta \cot \theta_{L} \right).
```

Here $q$ is the leaf-inclination-angle class and Tq is the total number of leaf-inclination-angle classes. Thus, if there are $18$ leaf-inclination-angle classes from $0◦$ to $90◦$ ($Tq = 18$), then each class consists of a $5◦$ interval. For example, $q = 1$, $q = 9$, and $q = 16$ include the angles from $0◦$ to $4◦$, $40◦$ to $44◦$, and $75◦$ to $79◦$, respectively. $g(q)$ is the distribution of the leaf-inclination-angle class $q$, which is a ratio of the leaf area belonging to class $q$ to total leaf area; $θ_{L}(q)$ is the midpoint angle of class $q$, which is the leaf-inclination angle used to represent class $q$.

This process is done trhough function `lad.Gtheta()`. In function `lad.alpha_k()` we compute $\alpha(\theta)$ for the median of $\theta$, the Beam Inclination Angles (BIA) with respect to zenith, in the Kth layer. We made use of the files `angles_<treename>.npy` and `weights_<treename>.npy` we store previously in the directory `lia` to get $g(q)$. The function `lad.alpha_k()` create three figures inside the `figures` directory:

- `alphas_<treename>_<voxel_size>.png`
- `bia_<treename>_<voxel_size>.png`
- `bia_per_k_<treename>_<voxel_size>.png`

Examples of this three figures can be found in Figures {numref}`alphasplot`, {numref}`biaplot`, and {numref}`biakplot` for a `downsample` of $5 \%$.

## Estimate LAD

Now that we have $\alpha(\theta)$ in the Kth layer (i.e. $\alpha(\theta, k)$), we can compute the LAI and therefore the LAD. We do this in function `lad.get_LADS()` which requires 4 input parameters:

- `m3att`: The numpy 3D-array attributes we derive in section {ref}`sec:ni`
- `voxel_size`: Voxel Size.
- `kbins`: $\Delta H$ in lengths if K.
- `alphas_k`: `lad.alpha_k()` function output.

This returns a numpy 2D-array with the height and LAD for the corresponding height with zero being the bottom of the PR.

Finally, with function `figures.plot_lads()` we plot LAD as a function of height for:

1. Using correction of $\alpha(\theta, K)$ taking the median of $\theta$ in the Kth layer.
2. Without $\alpha(\theta, K)$ correction
3. Truth LAD from mesh file.

The piece of code below we show all the above mentioned steps plus other minor steps. The output figure is saved in directory `figures` with name `LAD_<treename>_<voxel_size>.png` and shown in Fig. {numref}`ladplot`.

In [360]:
downsample = 0.05
voxel_size = 0.2

In [38]:
def runall(downsample, voxel_size):
    
    if downsample is not None:
        resdir = os.path.join(_data, mockname, 'lad_%s' %(str(downsample)))
        inds_file = os.path.join(resdir, 'inds.npy')
        inds = np.load(inds_file)
        print('downsample:', downsample)
    else:
        inds = np.ones(len(df), dtype=bool)
        resdir = os.path.join(_data, mockname, 'lad')

    isfigures = os.path.join(resdir, 'figures')
    if not os.path.exists(isfigures):
        os.makedirs(isfigures)

    print('voxel_size:', voxel_size)

    for key, val in trees.items():

        inPR = (val) & (leaves) & (inds)
        pointsPR = POINTS[inPR]
        sensorsPR = SENSORS[inPR]

        m3att = lad.compute_attributes(pointsPR, resdir, voxel_size, key)
        # get in down sample boolean array for LPC size
        inds_ = inds[(val) & (leaves)]
        lias, ws = lad.downsample_lia(mockname, key, inds_)
        voxk = lad.get_voxk(pointsPR, voxel_size)
        bia = lad.get_bia(pointsPR, sensorsPR)
        meshfile = lad.get_meshfile(mockname)

        figext = '%s_%s' %(key, str(voxel_size))
        # figext = None
        alphas_k = lad.alpha_k(bia, voxk, lias, ws, resdir, meshfile, figext=figext, 
                                klia=False, use_true_lia=True)

        kmax = m3att.shape[2]
        kbins = int(kmax/15)
        print(kbins)
        
        # lads_min = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,2], 1)
        # lads_max = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,4], 1)
        lads_mid = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,6], 1)
        lads_0 = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,6]*0+1, 1.0)
        lads_mesh = lad.get_LADS_mesh(meshfile, voxel_size, kbins, kmax)

        lads = {'Truth':lads_mesh, 'Correction Mean':lads_mid, 'No Correction':lads_0}
        clai = lad.get_clai(m3att, alphas_k)
        attributes_file = os.path.join(resdir, 'm3s_%s_%s.npy' %(key, str(voxel_size)))
        if os.path.isfile(attributes_file):
            RT = 'Y'
        else:
            RT = 'N'
            
        text = {'tree':key, 'VS':voxel_size, 'DS':downsample, 'RT':RT, 'CLAI':np.round(clai, 3)}
        txt = []
        for key, val in text.items():
            txt.append('%s=%s \n' %(key, str(val)))
        text = (' ').join(txt)

        savefig = os.path.join(resdir, 'figures','LAD_%s.png' %(figext))
        figures.plot_lads(lads, text, savefig=savefig)



In [46]:
downsample = None
voxel_size = 0.05

runall(downsample, voxel_size)

voxel_size: 0.05
max --> [189, 184, 119]
min --> [0, 0, 0]
No ray tracing for tree tree_0 and voxel size 0.050
foliage voxel dimensions: 	 (190, 185, 120)
ray tracker voxel dimensions: 	 (190, 185, 120)
Number of voxels ocupied by points cloud: 	 50318
Number of voxels ocupied by beam points cloud: 	 4218000
Total number of voxels in plant regions: 	 4218000
Number of voxels with attribute 1: 	 50318
Number of voxels with attribute 2: 	 4167682
Number of voxels with attribute 3: 	 0
8


In [20]:
for DS in [0.01, 0.05, 0.1, 0.15, 0.2, 0.3]:
    for VS in [0.05, 0.08]:

        get_rays(DS, VS)

inds file already exists for donwnsample of 0.010 at /Users/omar/projects/planttech/data/test/lad_0.01/inds.npy
max --> [178, 160, 114]
min --> [0, 0, 0]


1564it [01:13, 21.19it/s]


tot vox: 	 3314185
voxels hitted: 	 140412
Percentage of voxels hitted by beam: 0.04
voxels hitted (OLD): 	 141629
Percentage of voxels hitted by beam (OLD): 0.04
inds file already exists for donwnsample of 0.010 at /Users/omar/projects/planttech/data/test/lad_0.01/inds.npy
max --> [111, 100, 71]
min --> [0, 0, 0]


1564it [00:42, 37.01it/s]


tot vox: 	 814464
voxels hitted: 	 83432
Percentage of voxels hitted by beam: 0.10
voxels hitted (OLD): 	 85464
Percentage of voxels hitted by beam (OLD): 0.10
inds file already exists for donwnsample of 0.050 at /Users/omar/projects/planttech/data/test/lad_0.05/inds.npy
max --> [179, 182, 116]
min --> [0, 0, 0]


...

tot vox: 	 995448
voxels hitted: 	 703584
Percentage of voxels hitted by beam: 0.71
voxels hitted (OLD): 	 763198
Percentage of voxels hitted by beam (OLD): 0.77
inds file already exists for donwnsample of 0.300 at /Users/omar/projects/planttech/data/test/lad_0.3/inds.npy
max --> [187, 184, 119]
min --> [0, 0, 0]


40779it [1:12:41,  9.35it/s]


tot vox: 	 4173600
voxels hitted: 	 2476421
Percentage of voxels hitted by beam: 0.59
voxels hitted (OLD): 	 2596715
Percentage of voxels hitted by beam (OLD): 0.62
inds file already exists for donwnsample of 0.300 at /Users/omar/projects/planttech/data/test/lad_0.3/inds.npy
max --> [117, 115, 74]
min --> [0, 0, 0]


40779it [35:21, 19.22it/s]


tot vox: 	 1026600
voxels hitted: 	 788817
Percentage of voxels hitted by beam: 0.77
voxels hitted (OLD): 	 854805
Percentage of voxels hitted by beam (OLD): 0.83


In [7]:
for DS in [0.9]:
    for VS in [0.08, 0.1, 0.15, 0.2]:

        get_rays(DS, VS)

inds file already exists for donwnsample of 0.900 at /Users/omar/projects/planttech/data/test/lad_0.9/inds.npy
max --> [118, 115, 75]
min --> [0, 0, 0]


93644it [2:03:46, 12.61it/s] 


tot vox: 	 1049104
voxels hitted: 	 879681
Percentage of voxels hitted by beam: 0.84
voxels hitted (OLD): 	 961150
Percentage of voxels hitted by beam (OLD): 0.92
inds file already exists for donwnsample of 0.900 at /Users/omar/projects/planttech/data/test/lad_0.9/inds.npy
max --> [94, 92, 60]
min --> [0, 0, 0]


93644it [11:28:07,  2.27it/s] 


tot vox: 	 538935
voxels hitted: 	 450633
Percentage of voxels hitted by beam: 0.84
voxels hitted (OLD): 	 504497
Percentage of voxels hitted by beam (OLD): 0.94
inds file already exists for donwnsample of 0.900 at /Users/omar/projects/planttech/data/test/lad_0.9/inds.npy
max --> [63, 61, 40]
min --> [0, 0, 0]


93644it [1:21:26, 19.16it/s] 


tot vox: 	 162688
voxels hitted: 	 130686
Percentage of voxels hitted by beam: 0.80
voxels hitted (OLD): 	 155527
Percentage of voxels hitted by beam (OLD): 0.96
inds file already exists for donwnsample of 0.900 at /Users/omar/projects/planttech/data/test/lad_0.9/inds.npy
max --> [47, 46, 30]
min --> [0, 0, 0]


93644it [25:34, 61.02it/s] 


tot vox: 	 69936
voxels hitted: 	 54234
Percentage of voxels hitted by beam: 0.78
voxels hitted (OLD): 	 67534
Percentage of voxels hitted by beam (OLD): 0.97


In [538]:
downsample = 0.10
voxel_size = 0.2
sample = None
show = False

get_rays(downsample, voxel_size, sample, show)

inds file already exists for donwnsample of 0.100 at /Users/omar/projects/planttech/data/test/lad_0.1/inds.npy
max --> [47, 46, 29]
min --> [0, 0, 0]


14986it [03:46, 66.03it/s]

tot vox: 	 67680
voxels hitted: 	 52379
Percentage of voxels hitted by beam: 0.77
voxels hitted (OLD): 	 61933
Percentage of voxels hitted by beam (OLD): 0.92


In [539]:
runall(downsample, voxel_size)

downsample: 0.1
voxel_size: 0.2
max --> [47, 46, 29]
min --> [0, 0, 0]
foliage voxel dimensions: 	 (48, 47, 30)
ray tracker voxel dimensions: 	 (48, 47, 30)
Number of voxels ocupied by points cloud: 	 3496
Number of voxels ocupied by beam points cloud: 	 52379
Total number of voxels in plant regions: 	 67680
Number of voxels with attribute 1: 	 3496
Number of voxels with attribute 2: 	 52379
Number of voxels with attribute 3: 	 11805
2


qt.qpa.backingstore: Back buffer dpr of 2 doesn't match <_NSViewBackingLayer: 0x7fb2c9b9ca10> contents scale of 1 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 1 doesn't match <_NSViewBackingLayer: 0x7fb2c9b9ca10> contents scale of 2 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 1 doesn't match <_NSViewBackingLayer: 0x7fb2dbc1ae60> contents scale of 2 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 2 doesn't match <_NSViewBackingLayer: 0x7fb2dbc1ae60> contents scale of 1 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 2 doesn't match <_NSViewBackingLayer: 0x7fb2dbc1ae60> contents scale of 1 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 1 doesn't match <_NSViewBackingLayer: 0x7fb2dbc1ae60> contents scale of 2 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 1 doesn't match <_NSViewBackingLayer: 0x7fb2c9b9ca10> contents scale of 2 - updating layer to match.
qt.qpa.backin

In [ ]:
runall(downsample, voxel_size)

downsample: 0.05
voxel_size: 0.2
max --> [45, 46, 29]
min --> [0, 0, 0]
foliage voxel dimensions: 	 (46, 47, 30)
ray tracker voxel dimensions: 	 (46, 47, 30)
Number of voxels ocupied by points cloud: 	 2342
Number of voxels ocupied by beam points cloud: 	 48100
Total number of voxels in plant regions: 	 64860
Number of voxels with attribute 1: 	 2342
Number of voxels with attribute 2: 	 48100
Number of voxels with attribute 3: 	 14418
2


qt.qpa.backingstore: Back buffer dpr of 2 doesn't match <_NSViewBackingLayer: 0x7fb2dbc1ae60> contents scale of 1 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 1 doesn't match <_NSViewBackingLayer: 0x7fb2dbc1ae60> contents scale of 2 - updating layer to match.


## Ni and Np variation

In [8]:
results = []

for downsample in [0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 0.5, 0.9]:
    for voxel_size in [0.05, 0.08, 0.1, 0.15, 0.2]:

        if downsample is not None:
            resdir = os.path.join(_data, mockname, 'lad_%s' %(str(downsample)))
            inds_file = os.path.join(resdir, 'inds.npy')
            inds = np.load(inds_file)
            print('downsample:', downsample)
        else:
            inds = np.ones(len(df), dtype=bool)
            resdir = os.path.join(_data, mockname, 'lad')

        isfigures = os.path.join(resdir, 'figures')
        if not os.path.exists(isfigures):
            os.makedirs(isfigures)

        print('voxel_size:', voxel_size)

        for key, val in trees.items():

            inPR = (val) & (leaves) & (inds)
            pointsPR = POINTS[inPR]
            sensorsPR = SENSORS[inPR]

            m3att = lad.compute_attributes(pointsPR, resdir, voxel_size, key)

            results.append([downsample, voxel_size, (m3att == 1).sum(), (m3att == 2).sum(), (m3att == 3).sum()])

downsample: 0.01
voxel_size: 0.05
max --> [178, 160, 114]
min --> [0, 0, 0]
foliage voxel dimensions: 	 (179, 161, 115)
ray tracker voxel dimensions: 	 (179, 161, 115)
Number of voxels ocupied by points cloud: 	 670
Number of voxels ocupied by beam points cloud: 	 140412
Total number of voxels in plant regions: 	 3314185
Number of voxels with attribute 1: 	 670
Number of voxels with attribute 2: 	 140412
Number of voxels with attribute 3: 	 3173103
downsample: 0.01
voxel_size: 0.08
max --> [111, 100, 71]
min --> [0, 0, 0]
foliage voxel dimensions: 	 (112, 101, 72)
ray tracker voxel dimensions: 	 (112, 101, 72)
Number of voxels ocupied by points cloud: 	 663
Number of voxels ocupied by beam points cloud: 	 83432
Total number of voxels in plant regions: 	 814464
Number of voxels with attribute 1: 	 663
Number of voxels with attribute 2: 	 83432
Number of voxels with attribute 3: 	 730369
downsample: 0.01
voxel_size: 0.1
max --> [89, 80, 57]
min --> [0, 0, 0]
foliage voxel dimensions: 	 (

In [9]:
results = np.array(results)

In [10]:
total = np.array(np.sum(results[:,2:5], axis=1))
nI0 = results[:,2] / total
nP0 = results[:,3] / total
n00 = results[:,4] / total

nI = results[:,2]
nP = results[:,3]
n0 = results[:,4]


In [11]:
fig, (a0, a1) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [2, 1]}, figsize=(10,22))
colors = plt.cm.jet(np.linspace(0,1,5))

for num, voxel_size in enumerate([0.05, 0.08, 0.1, 0.15, 0.2]):

    keep = (results[:,1] == voxel_size)
    downsample = results[:,0][keep]
    a0.plot(downsample*100, nP0[keep], marker='*', color=colors[num], label='%s' %(voxel_size))
    a0.plot(downsample*100, nI0[keep], marker='*', ls='--', color=colors[num])
    a0.plot(downsample*100, n00[keep], marker='*', ls=':', color=colors[num])
    
    a1.plot(downsample*100, nI[keep]/(nI[keep]+nP[keep]), marker='*', ls='-', color=colors[num])

text = '$n_{P}(k)$: Solid \n $n_{I}(k)$: Dashed \n $n_{0}(k)$: Dotted'
props = dict(boxstyle='round', facecolor='green', alpha=0.3)
a0.text(20, 0.5, text, fontsize=14, bbox=props)
a0.legend(title='Voxel Size')
a0.set_ylabel(r'$N$', size=20)

a1.axhline(0.035, ls='--', c='k', lw=2)
a1.set_xlabel(r'Downsample (%)', size=20)
a1.set_ylabel(r'$n_{I}/(n_{I}+n_{P})$', size=20)
    


Text(0, 0.5, '$n_{I}/(n_{I}+n_{P})$')

## Densities in PR

In [231]:
downsample = 0.1
voxel_size = 0.15

In [35]:
def isinds(resdir, downsample):

    outdir = os.path.join(resdir, 'inds.npy')
    if os.path.exists(outdir):
        print('inds file already exists for donwnsample of %.3f at %s' %(downsample, outdir))

        inds = np.load(outdir)

        points = POINTS[inds]
        sensors = SENSORS[inds]

    else:

        print('inds not been created yet for donwnsample of %.3f' %(downsample))
        idx = np.random.randint(0, len(df), int(len(df) * downsample))
        inds = np.zeros(len(df), dtype=bool)
        inds[idx] = True

        points = POINTS[inds]
        sensors = SENSORS[inds]

        np.save(outdir, inds)

    return inds

def restmp(downsample, voxel_size, show=True):

    resdir = os.path.join(_data, mockname, 'lad_%s' %(str(downsample)))
    if not os.path.exists(resdir):
        os.makedirs(resdir)
    inds = isinds(resdir, downsample)

    for key, val in trees.items():

        inPR = (val) & (leaves) & (inds)
        pointsPR = POINTS[inPR]
        sensorsPR = SENSORS[inPR]

        density_overall, density, counts = lad.density_counts(pointsPR, voxel_size)
        
        dens_ratio = density / density_overall
        print('Mean density ration: %.2f' %(np.mean(dens_ratio)))
        print('Median density ration: %.2f' %(np.median(dens_ratio)))

        if show:

            fig = plt.figure(figsize=(10,6))
            plt.hist(counts, 25, align='left')
            plt.axvline(np.mean(counts), color='k', label='Mean counts = %.2f' %(np.mean(counts)))
            plt.xlabel(r'$n_{i}$', size=20)
            plt.legend()
            plt.show()


In [52]:
restmp(0.5, 0.05)

inds file already exists for donwnsample of 0.500 at /Users/omar/projects/planttech/data/test/lad_0.5/inds.npy
max --> [188, 183, 118]
min --> [0, 0, 0]
tot vox: 	 4138344
len points 26837
volume PR: 517.2930
volume voxel size: 0.0001
PR dimensions: 9.45, 9.20, 5.95
23659 2886 20773
Counts 26837
Density overal= 51.88
Mean density= 8895.69
Mean density considering all voxels= 50.28
Median density= 8000.00
Mean density ration: 171.47
Median density ration: 154.20


In [271]:
restmp(0.1, 0.15)

inds file already exists for donwnsample of 0.100 at /Users/omar/projects/planttech/data/test/lad_0.1/inds.npy
max --> [62, 61, 39]
min --> [0, 0, 0]
tot vox: 	 156240
len points 6389
volume PR: 527.3100
volume voxel size: 0.0034
PR dimensions: 9.45, 9.30, 6.00
4494 1338 3156
Counts 6389
Density overal= 12.12
Mean density= 398.08
Mean density considering all voxels= 11.15
Median density= 296.30
Mean density ration: 32.86
Median density ration: 24.45


In [275]:
restmp(0.2, 0.13)

inds file already exists for donwnsample of 0.200 at /Users/omar/projects/planttech/data/test/lad_0.2/inds.npy
max --> [72, 70, 45]
min --> [0, 0, 0]
tot vox: 	 238418
len points 12390
volume PR: 523.8043
volume voxel size: 0.0022
PR dimensions: 9.49, 9.23, 5.98
7679 2997 4682
Counts 12390
Density overal= 23.65
Mean density= 704.99
Mean density considering all voxels= 22.33
Median density= 455.17
Mean density ration: 29.80
Median density ration: 19.24


In [18]:
fig = plt.figure(figsize=(10,6))
plt.hist(dens_ratio, 25)
plt.axvline(np.mean(dens_ratio), color='k', label='Mean density = %.2f' %(np.mean(dens_ratio)))
plt.xlabel(r'$\eta_{i} = n_{i}/L^{3}$', size=20)
plt.legend()
plt.show()

NameError: name 'dens_ratio' is not defined

In [224]:
fig = plt.figure(figsize=(10,6))
plt.hist(density, 25)
plt.axvline(np.mean(density), color='k', label='Mean density = %.2f' %(np.mean(density)))
plt.xlabel(r'$\eta_{i} = n_{i}/L^{3}$', size=20)
plt.legend()
plt.show()